In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import axes3d  
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn import svm
import random
from imblearn.over_sampling import SMOTE, SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
# from EDA_helper import *
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.feature_selection import RFE, RFECV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
import torch.nn.functional as F
from torch.autograd import Variable

!pip install coral_pytorch
import coral_pytorch as coral
from coral_pytorch.layers import CoralLayer
from coral_pytorch.dataset import levels_from_labelbatch
from coral_pytorch.losses import coral_loss
from coral_pytorch.dataset import proba_to_label


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# Google Colab Instructions:



1.   Create folder "Honors Thesis"
2.   Add LSTM_cuda.ipynb to folder
3.   Create subfolder "datasets" containing the contents of the datasets folder from GitHub



In [3]:
USING_COLAB = True

In [2]:

if USING_COLAB:
  from google.colab import drive
  drive.mount('/drive')

KeyboardInterrupt: ignored

In [4]:
USE_GPU = True

dtype = torch.float32 

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print_every = 100

print('using device:', device)

using device: cuda


# Helper Functions

In [5]:
# @title
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import axes3d
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn import svm
import random
from imblearn.over_sampling import SMOTE, SVMSMOTE
# from EDA_helper import *
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.feature_selection import RFE, RFECV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline



#@title
def format_df(figs, ratings, to_drop = []):
    """
    figs: Pandas df, features
    ratings: Pandas df, labels
    to_drop: list, cols to remove

    """

    ratings1 = ratings[ratings['S&P Domestic Long Term Issuer Credit Rating'].notna()]
    ratings1 = ratings1[['Global Company Key','S&P Domestic Long Term Issuer Credit Rating', 'Data Date',
                        'Ticker Symbol']]

    print("Unique Companies in Ratings: ", len(ratings1["Ticker Symbol"].unique()))
    # print("Unique Companies in Features: ", len(figs["Ticker Symbol"].unique()))

    ## WE DON'T USE THIS
    if len(to_drop) == 0:
        figs_1 = figs.drop(list(figs.columns[[2, 3,4,11,12, 14, 16, 18, 20, 21, 23, 26, 30, 33, 35, 36, 37, 38]]), axis=1).dropna()
    else:
        # print(ratings1.shape, figs.shape)
        figs_1 = figs.drop(to_drop, axis=1).dropna()
        # print(ratings1.shape, figs_1.shape)
    print("Unique Companies in Features: ", len(figs_1["Ticker Symbol"].unique()))

    # lst3 = [value for value in ratings1["Ticker Symbol"].unique() if value in figs_1["Ticker Symbol"].unique()]


    companies = ratings1["Ticker Symbol"].unique()
    # companies
    # quarts = ['1231', '0331', '0630', '0930']
    # figs_1 = figs_1.loc[figs_1['Ticker Symbol'].isin(companies)]
    print("changed")
    figs_1['Data Date'] = figs_1['Data Date'].astype(str)
    ratings1['Data Date'] = ratings1['Data Date'].astype(str)
    # ratings1 = ratings1.loc[ratings1['Data Date'].str[-4:].isin(quarts)]
    return figs_1, ratings1


def join_numerical(figs_1, ratings1, numerical=[], ind_to_keep=[], years=[]):
    """
    in
    figs_1: Pandas df, features
    ratings1: Pandas df, labels
    numerical: List of str col. names of numerical columns to keep

    out
    x: np array
    y: np array
    merged: Pandas df
    """
    merged = pd.merge(figs_1, ratings1, how='inner', on=["Data Date", "Ticker Symbol"])
    merged = merged.rename(columns = {'S&P Domestic Long Term Issuer Credit Rating':'rating'})

    if len(numerical) == 0:
        numerical = ['Current Assets - Total', 'Cash','Long-Term Debt - Total',
        'Earnings Per Share (Basic) - Including Extraordinary Items - 12 Months Moving',
        'Invested Capital - Total - Quarterly','Inventories - Total', 'Pretax Income',
        'Operating Income Before Depreciation']
    vals = merged['Data Date'].str[:4]
    merged['year'] = vals
    # yrs =



    merged = merged[(merged['rating'] != 'D') & (merged['rating'] != 'SD')]

    counts =  merged['rating'].value_counts()
    good_classes = [counts.index[j] for j in range(len(counts)) if counts[j] >=7]
    merged = merged[merged['rating'].isin(good_classes)]
    if len(years) != 0:
        merged = merged[merged['year'].isin(years)]
    # merged = merged.drop(['year'], axis=1)
    if len(ind_to_keep) == 0:
        ind_to_keep = [True]*len(merged)

    x = merged.loc[ind_to_keep, numerical].values
    y = merged.loc[ind_to_keep,['rating']].values
    full = numerical.copy()
    full.append('rating')
    full.append('year')
    full.append('Ticker Symbol')



    merged = merged.loc[ind_to_keep, full]


    print("Intersection of Companies: ", len(merged['Ticker Symbol'].unique()))
    return x, y, merged

def encode(Y, form='', custom={}):

    if custom:
        pass

    else:
        if form == 'full':
            embed = {'BBB-':-1, 'BBB':0, 'A-':2, 'BBB+':1, 'AA+':7, 'AA':6, 'A':3, 'AA-':5, 'BB':-3, 'BB+':-2,
                                'AAA':8, 'B':-6, 'B+':-5, 'A+':4, 'BB-':-4, 'CCC+':-8,'CCC':-9,'CCC-':-10, 'CC+':-11,'CC':-12,
                                'CC-':-13,'B-':-7}
            labels = ['CCC','CCC+','B-', 'B', 'B+', 'BB-', 'BB', 'BB+', 'BBB-', 'BBB', 'BBB+', 'A-', 'A', 'A+',
                   'AA-', 'AA', 'AA+', 'AAA']
        elif form == 'stanfurd':
            embed = {'BBB-':-1, 'BBB':0, 'A-':2, 'BBB+':1, 'AA+':7, 'AA':6, 'A':3, 'AA-':5, 'BB':-3, 'BB+':-2,
                                'AAA':7, 'B':-6, 'B+':-5, 'A+':4, 'BB-':-4, 'CCC+':-8, 'B-':-7}
            labels = ['CCC+','B-', 'B', 'B+', 'BB-', 'BB', 'BB+', 'BBB-', 'BBB', 'BBB+', 'A-', 'A', 'A+',
                   'AA-', 'AA', 'AA+', 'AAA']
        elif form == 'three':
            embed = {'BBB-':1, 'BBB':1, 'A-':2, 'BBB+':1, 'AA+':2, 'AA':2, 'A':2, 'AA-':2, 'BB':0, 'BB+':0,
                    'AAA':2, 'B':0, 'B+':0, 'A+':2, 'BB-':0, 'CCC+':0, 'B-':0}
            labels = ['low', 'med', 'high']
        elif form == "IG":
            embed = {'BBB-':1, 'BBB':1, 'A-':1, 'BBB+':1, 'AA+':1, 'AA':1, 'A':1, 'AA-':1, 'BB':0, 'BB+':0,
                    'AAA':1, 'B':0, 'B+':0, 'A+':1, 'BB-':0, 'CCC+':0, 'B-':0}
            labels = ['HY', 'IG']

        elif form == 'letters':
            embed = {'BBB-':3, 'BBB':3, 'A-':4, 'BBB+':3, 'AA+':5, 'AA':5, 'A':4, 'AA-':5, 'BB':2, 'BB+':2,
                    'AAA':6, 'B':1, 'B+':1, 'A+':4, 'BB-':2, 'CCC+':0, 'B-':1}
            labels = ['CCC', 'B', 'BB', 'BBB', 'A', 'AA','AAA']
        # elif form == 'letters ex CCC':
        else:
            embed = {'BBB-':4, 'BBB':4, 'A-':5, 'BBB+':4, 'AA+':6, 'AA':6, 'A':5, 'AA-':6, 'BB':3, 'BB+':3,
                  'AAA':7, 'B':2, 'B+':2, 'A+':5, 'BB-':3, 'CCC+':1, 'B-':2, 'CCC':1,'CCC-':1, 'CC+':0,'CC':0,
                    'CC-':0}
            labels = ['B', 'BB', 'BBB', 'A', 'AA','AAA']
            # colors = ['#630C3A', '#39C8C6', '#D3500C', '#FFB139', '#828282', '#17d8ff', '#1770ff']
        Y_emb = np.array([embed[i] for i in Y.T[0]]).ravel()

    return Y_emb, labels

def decode(Y, form='', custom={}):
    if form == 'full':
            embed = {-1:'BBB-', 0:'BBB', 2:'A-', 1:'BBB+', 7:'AA+', 6:'AA', 3:'A', 5:'AA-', -3:'BB', -2:'BB+',
                            8:'AAA', -6:'B', -5:'B+', 4:'A+', -4:'BB-', -8:'CCC+',-9:'CCC',-10:'CCC-', -11:'CC+',
                     -12:'CC', -13:'CC-',-7:'B-'}
            labels = ['CC','CC+','CCC', 'CCC+','B-', 'B', 'B+', 'BB-', 'BB', 'BB+', 'BBB-', 'BBB', 'BBB+', 'A-', 'A', 'A+',
                   'AA-', 'AA', 'AA+', 'AAA']
    Y_emb = np.array([embed[i] for i in Y.T[0]]).ravel()
    return Y_emb, labels
def data_split(X, Y,full=None, split=[0.7, 0.1], method="numer", smote=False, seed=100, neigh=5):
    """
    split = [ratio of train, ratio of val]
    note: ratio of test = 1.0 - (ratio of train + ratio of val)
    """
    np.random.seed(seed)

    #### TODO: if smote, give a non-smote Validation set
    ## No Validation set
    if split[1] == 0.0:
        test_split = 1.0 - split[0]

        if method == 'numer':
            splits = int(np.round(split[0]*X.shape[0]))
            cc = list(np.arange(X.shape[0])).copy()
            np.random.shuffle(cc)
            train_inds = cc[:splits]
            test_inds = cc[splits:]
            X_train = X[train_inds, :]
            Y_train = Y[train_inds]
            X_test =  X[test_inds, :]
            Y_test =  Y[test_inds]

            X_val = []
            Y_val = []

        # elif method == "holdout":
            # X is DataFrame
            # Y is empty
        if smote:
            # ks = np.min([np.min(), 5])
            dd = {}
            uu = {}
            kounts = pd.Series(Y_train).value_counts().to_dict()
            max_val = max(kounts.values())
            for k,v in kounts.items():
              dd[k] = int((max_val - v)*0.2 + v)
            for c,d in sorted(kounts.items(), key=lambda item: item[1])[-5:]:
              uu[c] = int(d*0.7)

            oversample = SMOTE(sampling_strategy=dd, k_neighbors=neigh)
            undersample =RandomUnderSampler(sampling_strategy=uu)
            steps = [('o', oversample), ('u', undersample)]
            pipeline = Pipeline(steps=steps)
            # print("SVMMMM")

            X_train, Y_train = pipeline.fit_resample(X_train, Y_train)


    else:
        if method == 'numer':
            splits = int(np.round(split[0]*X.shape[0]))
            splits1 = splits + int(np.round(split[1]*X.shape[0]))
            cc = list(np.arange(X.shape[0])).copy()
            np.random.shuffle(cc)
            train_inds = cc[:splits]
            val_inds = cc[splits:splits1]
            test_inds = cc[splits1:]

            X_train = X[train_inds, :]
            Y_train = Y[train_inds]
            X_val = X[val_inds, :]
            Y_val = Y[val_inds]

            X_test =  X[test_inds, :]
            Y_test =  Y[test_inds]

        if smote:
            # ks = np.min([np.min(), 5])
            oversample = SMOTE(k_neighbors=neigh)
            X_train, Y_train = oversample.fit_resample(X_train, Y_train)



    return X_train, Y_train, X_val, Y_val, X_test, Y_test


In [6]:
def import_data():
  if USING_COLAB:
    ratings = pd.read_excel("drive/My Drive/Honors Thesis/datasets/ratings_all_energ_new.xlsx")
    ratio = pd.read_excel("drive/My Drive/Honors Thesis/datasets/ratios_2_all_energ_new.xlsx")
    energ = pd.read_excel("drive/My Drive/Honors Thesis/datasets/energ_specific_all_new.xlsx")
  else:
    ratings = pd.read_excel("../datasets/ratings_all_energ_new.xlsx")
    ratio = pd.read_excel("../datasets/ratios_2_all_energ_new.xlsx")
    energ = pd.read_excel("../datasets/energ_specific_all_new.xlsx")
  cols = ratio.columns
  ratio = ratio.drop(cols[0], axis=1)
  ratio = ratio.rename(columns = {'Public Date':'Data Date','EXCHANGE TICKER SYMBOL - HISTORICAL':'Ticker Symbol'})

  ratio1, ratings1 = format_df(ratio.copy(), ratings.copy(), 
                              ['Trailing P/E to Growth (PEG) ratio','Dividend Yield', 
                                'Interest/Average Total Debt', 'Free Cash Flow/Operating Cash Flow'])

  numer = list(ratio1.columns[4:-1])

  cols = energ.columns
  aa = cols[[0,1,2,3,4,5,6, 10,11,14,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35]]
  energ1 = energ.drop(
        aa, 
      axis=1)

  energ1['Data Date'] = energ1['Data Date'].astype(str)

  features = pd.merge(ratio1, energ1,how='left',on=["Data Date", "Ticker Symbol"])
  lst3 = [value for value in ratio["Ticker Symbol"].unique() if value in energ["Ticker Symbol"].unique()]

  print(len(ratio["Ticker Symbol"].unique()), len(energ["Ticker Symbol"].unique()), 
        len(ratio1["Ticker Symbol"].unique()), len(energ1["Ticker Symbol"].unique()))

  first_col = features.pop("Ticker Symbol") 
  ff = len(features.columns)
  features.insert(ff, "Ticker Symbol", first_col)
  numer = list(features.columns[4:-1])

  X1,Y1,full1 = join_numerical(features, ratings1, numerical=numer, years=['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017'])
  return X1,Y1,full1

In [7]:
def sequencize(X1, Y1, full1, win=6):
  """
  Splits data into windows for LSTM training.
  For use with random split strategy
  win: sequence length
  Output X: (N*, win, features)
         Y: (N*, win, 1)
  """
  full1 = full1.reset_index()
  comps = list(full1['Ticker Symbol'].unique())
  labout = []
  inout = []
  t_labout = []
  t_inout = []
  diff_in = []
  diff_lab = []

  for c in comps:
    curr_in = full1.index[full1['Ticker Symbol'] == c]

    indXd = X1[curr_in,:]
    indYd = Y1[curr_in]
    V = len(indXd)
    # wins = [V-win - i for i in range(win)]
    for i in range(V % win, V):
      t_seq = indXd[i:i+win,:]
      t_lab = indYd[i:i+win]

      if t_seq.shape[0] == win:
        # if np.all(t_lab == t_lab[0]) == False:
        #     diff_in.append(t_seq)
        #     diff_lab.append(t_lab)
        #     t_inout.append(t_seq)
        #     t_labout.append(t_lab)
        inout.append(t_seq)
        labout.append(t_lab)

        
  # print(np.stack(inout).shape)
  ins = np.transpose(np.dstack(inout), (2, 0, 1))
  labs = np.transpose(np.dstack(labout), (2, 0, 1))

  assert ins.shape[0] == labs.shape[0]
  assert ins.shape[1] == labs.shape[1] 
  return ins, labs


In [8]:
def val_sequencize(X1, Y1, full1, win=6):
  """
  win: sequence length
  Output X: (N*, win, features)
         Y: (N*, win, 1)
  """
  full1 = full1.reset_index()
  comps = list(full1['Ticker Symbol'].unique())
  labout = []
  inout = []
  t_labout = []
  t_inout = []

  v_labout = []
  v_inout = []

  diff_in = []
  diff_lab = []

  for c in comps:
    curr_in = full1.index[full1['Ticker Symbol'] == c]

    indXd = X1[curr_in,:]
    indYd = Y1[curr_in]
    V = len(indXd)
    # test_wins = [V-win - j for j in range(win)]
    # val_wins = [V-3*win - k for k in range(win)]
    for i in range(V % win, V):
      t_seq = indXd[i:i+win,:]
      t_lab = indYd[i:i+win]

      if t_seq.shape[0] == win:
        # print(t_seq.shape)
        # if np.all(t_lab == t_lab[0]) == False:
        #     diff_in.append(t_seq)
        #     diff_lab.append(t_lab)
        #     t_inout.append(t_seq)
        #     t_labout.append(t_lab)
          
        if i in range(V-2*win, V-win):
        #   if np.all(t_lab == t_lab[0]) == False:
          diff_in.append(t_seq)
          diff_lab.append(t_lab)
          t_inout.append(t_seq)
          t_labout.append(t_lab)
          # print(t_seq.shape, i, V - win)
        elif i in range(V-4*win, V-2*win):
          v_inout.append(t_seq)
          v_labout.append(t_lab)

        else:
          inout.append(t_seq)
          labout.append(t_lab)

  ins = np.transpose(np.dstack(inout), (2, 0, 1))
  labs = np.transpose(np.dstack(labout), (2, 0, 1))
  t_ins = np.transpose(np.dstack(t_inout), (2, 0, 1))
  t_labs = np.transpose(np.dstack(t_labout), (2, 0, 1))
  d_ins = np.transpose(np.dstack(diff_in), (2, 0, 1))
  d_labs = np.transpose(np.dstack(diff_lab), (2, 0, 1))

  v_ins = np.transpose(np.dstack(v_inout), (2, 0, 1))
  v_labs = np.transpose(np.dstack(v_labout), (2, 0, 1))
  # print(d_ins.shape, d_labs.shape)


  assert ins.shape[0] == labs.shape[0]
  assert ins.shape[1] == labs.shape[1] 
  return ins, labs, t_ins, t_labs, v_ins, v_labs


In [9]:
def plotGD(losses, model):
  plt.plot(losses)
  plt.ylabel('Loss')
  plt.xlabel('Iterations (in 100s)')
  plt.title(model + " Gradient Descent (ADAM)")
def plotAcc(vals, model):
  plt.plot(vals)
  plt.ylabel('Accuracy')
  plt.xlabel('Iterations (in 100s)')
  plt.title(model +  " Gradient Descent (ADAM)")

# LSTM data processing


In [10]:
def rand_split(X1, Y1, full1, win=6):
  win=win
  X1,Y1 = sequencize(X1, Y1, full1, win=win)

  X1, X_hold_,Y1, Y_hold_ = train_test_split(X1, Y1, test_size=0.2, random_state=101)

  scl_X = np.zeros_like(X1)
  scalers = {}
  enc_Y = np.zeros_like(Y1)
  for i in range(X1.shape[1]):
      scalers[i] = StandardScaler()
      scl_X[:, i, :] = scalers[i].fit_transform(X1[:, i, :]) 
      enc_Y[:, i, :] = encode(Y1[:,i,:], 'full')[0][:, np.newaxis] 

  scl_X = np.nan_to_num(scl_X)

  X_hold = np.zeros_like(X_hold_)
  Y_hold = np.zeros_like(Y_hold_)
  for j in range(X_hold_.shape[1]):
      X_hold[:, j, :] = scalers[j].transform(X_hold_[:, j, :]) 
      Y_hold[:, j, :] = encode(Y_hold_[:, j, :], 'full')[0][:, np.newaxis]
 
  X_hold = np.nan_to_num(X_hold)

  minny = np.min((np.min(enc_Y), np.min(Y_hold)))

  enc_Y += np.abs(minny)
  Y_hold += np.abs(minny)
  return scl_X, enc_Y, X_hold, Y_hold


In [11]:
def time_split(X1, Y1, full1, win=6):
  win=6
  X1,Y1, X_hold_, Y_hold_, X_val, Y_val = val_sequencize(X1, Y1, full1, win=win)
  scl_X = np.zeros_like(X1)
  val_X = np.zeros_like(X_val)
  scalers = {}
  enc_Y = np.zeros_like(Y1)
  val_Y = np.zeros_like(Y_val)
  for i in range(X1.shape[1]):
      scalers[i] = StandardScaler()
      scl_X[:, i, :] = scalers[i].fit_transform(X1[:, i, :]) 
      enc_Y[:, i, :] = encode(Y1[:,i,:], 'full')[0][:, np.newaxis] 

  for z in range(X_val.shape[1]):
      val_X[:, z, :] = scalers[z].transform(X_val[:, z, :]) 
      val_Y[:, z, :] = encode(Y_val[:,z,:], 'full')[0][:, np.newaxis] 
  scl_X = np.nan_to_num(scl_X)
  val_X = np.nan_to_num(val_X) 

  X_hold = np.zeros_like(X_hold_)
  Y_hold = np.zeros_like(Y_hold_)

  for j in range(X_hold_.shape[1]):
      X_hold[:, j, :] = scalers[j].transform(X_hold_[:, j, :]) 
      Y_hold[:, j, :] = encode(Y_hold_[:, j, :], 'full')[0][:, np.newaxis]
  X_hold = np.nan_to_num(X_hold)

  minny = np.min((np.min(enc_Y), np.min(Y_hold), np.min(val_Y)))
  enc_Y += np.abs(minny)
  val_Y += np.abs(minny)
  Y_hold += np.abs(minny)

  return scl_X, enc_Y, val_X, val_Y, X_hold, Y_hold

In [12]:
def lstm_torch(X_train, Y_train, X_val, Y_val, X_test, Y_test):
  Y_train_, Y_val_ = Y_train.astype('int32'), Y_val.astype('int32')
  Xt_train = torch.FloatTensor(X_train)
  Yt_train = torch.Tensor(Y_train_)
  Xt_val = torch.FloatTensor(X_val)
  Yt_val = torch.Tensor(Y_val_)

  Xt_test = torch.FloatTensor(X_test)
  Yt_test = torch.Tensor(Y_test.astype('int32'))

  train_set = TensorDataset(Xt_train, Yt_train)
  loader_train = DataLoader(train_set, batch_size=10)

  val_set = TensorDataset(Xt_val, Yt_val)
  loader_val = DataLoader(val_set, batch_size=10)

  test_set = TensorDataset(Xt_test, Yt_test)
  loader_test = DataLoader(test_set, batch_size=10)
  return loader_train, loader_val, loader_test


# Train/Eval Functions

In [13]:
def validate(loader, model, mode = 0):

    num_correct = 0
    num_samples = 0
    last_corr = 0
    last_samples = 0
    mae = 0.
    model.eval()  
    with torch.no_grad():
 
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  
            y = y.float().to(device=device)
            
            if mode == 2:
              scores = model(x)
              _, preds = scores.max(1)
              y = torch.squeeze(y)
            elif mode == 3:
              scores, probs = model(x)
              # _, preds = scores.max(1)
              preds = proba_to_label(probs).float()
              # y = torch.squeeze(y)
              y = torch.flatten(y.view(y.size(0), -1).int())

            num_correct += (preds == y).sum()
            # if y[:, -1] != y[:, -1]: 

            if mode == 2:
              num_samples += preds.size(0) * preds.size(1)
              # last_corr += (preds[:, -1] == y[:, -1]).sum()
              # last_samples += preds.size(0)            

            else:
              num_samples += preds.size(0)
            mae += torch.sum(torch.abs(preds - y))
            # mse += torch.sum((preds - y)**2)
        acc = float(num_correct) / num_samples
        # if mode ==2:
        #   last_acc = float(last_corr) / last_samples
        #   print(last_acc)
        mae = mae / num_samples
        # mse = mse / num_examples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc), f'Mean absolute error: {mae:.2f}')
    return acc, mae

In [14]:
def train_func(model, optimizer, epochs=1, mode=0):
    """
    Inputs:
    - model: A PyTorch model.
    - optimizer: Optimizer object
    - epochs: int  
    Returns: Lists of validation accuracies at the end of each epoch.
    """
    print(device)
    model = model.to(device=device)  
    train_accs = []
    val_accs = []
    val_mae = []
    losses = []
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  
           
            x = x.to(device=device, dtype=dtype) 

            # LSTM  
            if mode == 2:
              ## LSTM Mode
              y = y.to(device=device, dtype=torch.long)
              y = torch.squeeze(y)
              scores = model(x)

              # print(scores.size(), y.size())
              loss = F.cross_entropy(scores, y)
            # LSTM-OR
            elif mode == 3:
              # print(torch.flatten(y.view(y.size(0), -1).int()).size())
              # print("nums :", num_classes)
              levels = levels_from_labelbatch(torch.flatten(y.view(y.size(0), -1).int()), num_classes=num_classes)
              # print('lev: ', levels.size())
              levels = levels.to(device=device, dtype=torch.long)
              # print("levels: ",levels.size())
              scores, probs = model(x)
              # print('s:',scores.size(), 'lev: ', levels.size())
              loss = coral_loss(scores, levels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                losses.append(loss.item())
                validate(loader_val, model, mode)
                print()
        acc_, mae_ = validate(loader_val, model, mode)
        val_accs.append(acc_)
        val_mae.append(mae_)
        
    return val_accs, val_mae, losses

# Base LSTM


In [15]:
class BASELSTM(nn.Module):
    """."""

    def __init__(self, input_dimensions=4, hidden_dimensions=512, num_classes=18, num_layers=1,
                 dropout=0.0, batch_first=True, bidirectional=False, seq_len=6):
        super(BASELSTM, self).__init__()
        
        self.input_dimensions = input_dimensions
        self.hidden_dimensions = hidden_dimensions

        self.num_classes = num_classes

        self.num_layers = num_layers
        self.dropout = dropout

        self.batch_first = batch_first

        self.bidirectional = bidirectional
        self.seq_len = seq_len

        self.lstm = nn.LSTM(input_size=self.input_dimensions, hidden_size=self.hidden_dimensions,
                          num_layers=self.num_layers, batch_first=self.batch_first,
                          dropout=self.dropout, bidirectional=self.bidirectional)
        self.fc = nn.Linear(in_features=self.hidden_dimensions, out_features=self.num_classes)
        
    def forward(self, X):
        batch_index = 0 if self.batch_first else 1
        num_direction = 2 if self.bidirectional else 1
        
        lstmh_0 = torch.zeros(self.num_layers * num_direction, 
                       X.size(batch_index), self.hidden_dimensions)
        # print(h_0.size())
        lstmh_0 = lstmh_0.to(device)

        c_0 = torch.zeros(self.num_layers * num_direction, 
                       X.size(batch_index), self.hidden_dimensions)
        c_0 = c_0.to(device)

        output_lstm, h_n = self.lstm(X, (lstmh_0, c_0))

        # output_lstm has shape (batch_size, seq_len, hidden_dimensions)

        fc_output = self.fc(output_lstm)
        
        fc_output = fc_output.permute(0,2,1)
        # print(fc_output.size())
        # print(fc_output.size())
        # fc_output = fc_output.reshape(fc_output.size()[0], fc_output.size()[1])
        # fc_output = fc_output.view(-1, fc_output.size()[-1])
        # fc_output will be batch_size*seq_len*num_classes
        return fc_output 

# LSTM-OR [Vishnu et al., 2019]

In [16]:
class LSTM_OR(nn.Module):
    """."""

    def __init__(self, input_dimensions=4, hidden_dimensions=512, num_classes=18, num_layers=1,
                 dropout=0.0, batch_first=True, bidirectional=False, seq_len=6):
        super(LSTM_OR, self).__init__()
        
        self.input_dimensions = input_dimensions
        self.hidden_dimensions = hidden_dimensions

        self.num_classes = num_classes

        self.num_layers = num_layers
        self.dropout = dropout

        self.batch_first = batch_first

        self.bidirectional = bidirectional
        self.seq_len = seq_len

        self.lstm = nn.LSTM(input_size=self.input_dimensions, hidden_size=self.hidden_dimensions,
                          num_layers=self.num_layers, batch_first=self.batch_first,
                          dropout=self.dropout, bidirectional=self.bidirectional)
        
        self.fc = nn.Linear(in_features=self.hidden_dimensions, out_features=self.num_classes)
        
    def forward(self, X):
        batch_index = 0 if self.batch_first else 1
        num_direction = 2 if self.bidirectional else 1
          
        lstmh_0 = torch.zeros(self.num_layers * num_direction, 
                       X.size(batch_index), self.hidden_dimensions)
        lstmh_0 = lstmh_0.to(device)

        c_0 = torch.zeros(self.num_layers * num_direction, 
                       X.size(batch_index), self.hidden_dimensions)
        c_0 = c_0.to(device)
        # X = X.permute(0, 2, 1)
        output_lstm, h_n = self.lstm(X, (lstmh_0, c_0))      
  
        output_lstm = torch.reshape(output_lstm, (output_lstm.size(0)*output_lstm.size(1),output_lstm.size(2)))
        fc_output = self.fc(output_lstm)

        probs = torch.sigmoid(fc_output)
        return fc_output, probs 



# Randomly Split Data


In [17]:
X1, Y1, full1 = import_data()
win=6
scl_X, enc_Y, X_hold, Y_hold = rand_split(X1, Y1, full1, win=6)
num_classes = len(np.unique(enc_Y))
num_features = scl_X.shape[2]
X_train, Y_train, _, _,X_val, Y_val = data_split(scl_X, enc_Y, split=[0.8, 0.0], smote=False, seed = 100, neigh=4)

loader_train, loader_val, loader_test = lstm_torch(X_train, Y_train, X_val, Y_val, X_hold, Y_hold)

Unique Companies in Ratings:  335
Unique Companies in Features:  253
changed
273 819 253 819
Intersection of Companies:  97


In [18]:
lr = 1e-3
hidden_layer_size = 250 
epochs = 100

In [19]:
learning_rate = 1e-3
model = BASELSTM(input_dimensions=num_features, hidden_dimensions=hidden_layer_size, num_classes=num_classes, num_layers=1,
                 dropout=0.3, batch_first=True, bidirectional=False, seq_len=win)

mode = 2
# optimizer = optim.SGD(model.parameters(), lr=learning_rate,
#                      momentum=0.9, nesterov=True)  
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  

validation_accs, maes, losses = train_func(model, optimizer, epochs=60, mode=mode)
print("Final Acc: \n")
validate(loader_val, model, mode)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


cuda
Iteration 0, loss = 2.8906
Got 349 / 4584 correct (7.61) Mean absolute error: 3.23

Iteration 100, loss = 1.6132
Got 1423 / 4584 correct (31.04) Mean absolute error: 2.05

Iteration 200, loss = 2.0323
Got 1775 / 4584 correct (38.72) Mean absolute error: 1.82

Iteration 300, loss = 1.9432
Got 1998 / 4584 correct (43.59) Mean absolute error: 1.70

Got 1943 / 4584 correct (42.39) Mean absolute error: 1.69
Iteration 0, loss = 2.0273
Got 1937 / 4584 correct (42.26) Mean absolute error: 1.68

Iteration 100, loss = 0.9849
Got 2145 / 4584 correct (46.79) Mean absolute error: 1.52

Iteration 200, loss = 1.5970
Got 2272 / 4584 correct (49.56) Mean absolute error: 1.39

Iteration 300, loss = 1.4147
Got 2441 / 4584 correct (53.25) Mean absolute error: 1.30

Got 2351 / 4584 correct (51.29) Mean absolute error: 1.35
Iteration 0, loss = 1.7285
Got 2348 / 4584 correct (51.22) Mean absolute error: 1.34

Iteration 100, loss = 0.8069
Got 2603 / 4584 correct (56.78) Mean absolute error: 1.16

Iterati

(0.9380453752181501, tensor(0.1307, device='cuda:0'))

In [21]:
validate(loader_test, model, mode)

Got 5383 / 5730 correct (93.94) Mean absolute error: 0.13


(0.9394415357766143, tensor(0.1286, device='cuda:0'))

In [22]:
ordinal_out = num_classes - 1 
model = LSTM_OR(input_dimensions=num_features, hidden_dimensions=hidden_layer_size, num_classes=ordinal_out, num_layers=1,
                 dropout=0.0, batch_first=True, bidirectional=False, seq_len=win)
mode = 3
optimizer = optim.Adam(model.parameters(), lr=lr)  

validation_accs, maes, losses = train_func(model, optimizer, epochs=epochs, mode=mode)
print("Final Acc: \n")
validate(loader_val, model, mode)

cuda
Iteration 0, loss = 11.7966
Got 520 / 4584 correct (11.34) Mean absolute error: 2.87

Iteration 100, loss = 4.7213
Got 781 / 4584 correct (17.04) Mean absolute error: 2.04

Iteration 200, loss = 6.0165
Got 847 / 4584 correct (18.48) Mean absolute error: 1.90

Iteration 300, loss = 5.8886
Got 929 / 4584 correct (20.27) Mean absolute error: 1.76

Got 1055 / 4584 correct (23.01) Mean absolute error: 1.84
Iteration 0, loss = 4.6014
Got 1041 / 4584 correct (22.71) Mean absolute error: 1.86

Iteration 100, loss = 2.8745
Got 994 / 4584 correct (21.68) Mean absolute error: 1.73

Iteration 200, loss = 5.0593
Got 1239 / 4584 correct (27.03) Mean absolute error: 1.57

Iteration 300, loss = 5.7014
Got 1446 / 4584 correct (31.54) Mean absolute error: 1.49

Got 1406 / 4584 correct (30.67) Mean absolute error: 1.53
Iteration 0, loss = 4.0623
Got 1387 / 4584 correct (30.26) Mean absolute error: 1.53

Iteration 100, loss = 2.0211
Got 1382 / 4584 correct (30.15) Mean absolute error: 1.49

Iteration

(0.924956369982548, tensor(0.1215, device='cuda:0'))

In [23]:
validate(loader_test, model, mode)

Got 5302 / 5730 correct (92.53) Mean absolute error: 0.12


(0.925305410122164, tensor(0.1197, device='cuda:0'))

# Time Series Split

In [42]:
X1, Y1, full1 = import_data()
win=6
X_train, Y_train, X_val, Y_val, X_test, Y_test = time_split(X1, Y1, full1, win=6)
num_classes = len(np.unique(Y1))
num_features = X_train.shape[2]

loader_train, loader_val, loader_test = lstm_torch(X_train, Y_train, X_val, Y_val, X_test, Y_test)

Unique Companies in Ratings:  335
Unique Companies in Features:  253
changed
273 819 253 819
Intersection of Companies:  97


In [43]:
lr = 1e-3
hidden_layer_size = 250 
epochs = 100

In [44]:
model = BASELSTM(input_dimensions=num_features, hidden_dimensions=hidden_layer_size, num_classes=num_classes, num_layers=1,
                 dropout=0.0, batch_first=True, bidirectional=False, seq_len=win)

mode = 2
# optimizer = optim.SGD(model.parameters(), lr=learning_rate,
#                      momentum=0.9, nesterov=True)  
optimizer = optim.Adam(model.parameters(), lr=lr)  

validation_accs, maes, losses = train_func(model, optimizer, epochs=epochs, mode=mode)
print("Final Acc: \n")
validate(loader_val, model, mode)

cuda
Iteration 0, loss = 2.8461
Got 170 / 5904 correct (2.88) Mean absolute error: 6.31

Iteration 100, loss = 5.8868
Got 734 / 5904 correct (12.43) Mean absolute error: 3.44

Iteration 200, loss = 2.4542
Got 1250 / 5904 correct (21.17) Mean absolute error: 2.91

Iteration 300, loss = 1.7790
Got 1302 / 5904 correct (22.05) Mean absolute error: 2.63

Got 1260 / 5904 correct (21.34) Mean absolute error: 2.61
Iteration 0, loss = 3.4930
Got 1261 / 5904 correct (21.36) Mean absolute error: 2.58

Iteration 100, loss = 3.9356
Got 896 / 5904 correct (15.18) Mean absolute error: 3.25

Iteration 200, loss = 2.1277
Got 1430 / 5904 correct (24.22) Mean absolute error: 2.74

Iteration 300, loss = 1.5277
Got 1394 / 5904 correct (23.61) Mean absolute error: 2.44

Got 1457 / 5904 correct (24.68) Mean absolute error: 2.23
Iteration 0, loss = 3.6069
Got 1424 / 5904 correct (24.12) Mean absolute error: 2.24

Iteration 100, loss = 3.0251
Got 1028 / 5904 correct (17.41) Mean absolute error: 3.10

Iteration

(0.5164295392953929, tensor(1.3579, device='cuda:0'))

In [45]:
validate(loader_test, model, mode)

Got 1972 / 3348 correct (58.90) Mean absolute error: 1.18


(0.5890083632019116, tensor(1.1759, device='cuda:0'))

In [46]:
ordinal_out = num_classes - 1 
model = LSTM_OR(input_dimensions=num_features, hidden_dimensions=hidden_layer_size, num_classes=ordinal_out, num_layers=1,
                 dropout=0.0, batch_first=True, bidirectional=False, seq_len=win)
mode = 3
optimizer = optim.Adam(model.parameters(), lr=lr)  

validation_accs, maes, losses = train_func(model, optimizer, epochs=epochs, mode=mode)
print("Final Acc: \n")
validate(loader_val, model, mode)

cuda
Iteration 0, loss = 11.8245
Got 532 / 5904 correct (9.01) Mean absolute error: 3.02

Iteration 100, loss = 5.1343
Got 765 / 5904 correct (12.96) Mean absolute error: 3.31

Iteration 200, loss = 8.9652
Got 1057 / 5904 correct (17.90) Mean absolute error: 2.74

Iteration 300, loss = 3.3718
Got 1106 / 5904 correct (18.73) Mean absolute error: 2.48

Got 625 / 5904 correct (10.59) Mean absolute error: 2.42
Iteration 0, loss = 2.5734
Got 548 / 5904 correct (9.28) Mean absolute error: 2.42

Iteration 100, loss = 5.1131
Got 771 / 5904 correct (13.06) Mean absolute error: 3.19

Iteration 200, loss = 3.8991
Got 1047 / 5904 correct (17.73) Mean absolute error: 2.14

Iteration 300, loss = 2.4993
Got 1027 / 5904 correct (17.39) Mean absolute error: 2.28

Got 686 / 5904 correct (11.62) Mean absolute error: 2.26
Iteration 0, loss = 2.3384
Got 641 / 5904 correct (10.86) Mean absolute error: 2.26

Iteration 100, loss = 4.9452
Got 895 / 5904 correct (15.16) Mean absolute error: 2.93

Iteration 200,

(0.4847560975609756, tensor(1.1260, device='cuda:0'))

In [48]:
validate(loader_test, model, mode)

Got 1694 / 3348 correct (50.60) Mean absolute error: 1.16


(0.505973715651135, tensor(1.1556, device='cuda:0'))